In [1]:
import re
import sys
import json
import numpy as np
import pandas as pd
from pprint import pprint

In [2]:
with open('recipe_collection.json', 'r') as f:
    data = json.load(f)
len(data)

1827

In [3]:
labels = []
for record in data:
    labels.append(data[record]['label'])

In [302]:
ingredient_lists = []
uris = []

for record in data:
    uris.append(record)
    ingredient_lists.append(data[record]['ingredientLines'])

In [303]:
len(labels), len(ingredient_lists)

(1827, 1827)

In [304]:
scrubbed_ingredients = []
measurers = re.compile(r' (cups|cup|kg|tsp|sp|ounces|grams|tb|gram|ounce|mililiters|qt|tablespoon|tablespoons|teaspoon|tbs|tbsp|oz|can|large|small|pinch|bag|lb|ml|tub) ')
descriptors = re.compile('(slice[d]?|about|finely|chopped|new|all-purpose|all purpose|thinly|thin|plus|optionalcrushed|crumbles|ground|minced|powder|chopped|sliced)')
ammounts = re.compile(r'\d+(\/\d+|\.\d+)?') # TODO use these to weight vector averaging?
stopwords = re.compile(r' (or|the|other|of|to|on|for) ')
nonascii = re.compile(r'[^\x00-\x7F]+')
multiplespaces = re.compile(' +')

for ingredient_list in ingredient_lists:
    scrubbed_lines = []
    for line in ingredient_list:
        line = ' ' + line.lower() + ' '
        
        line = line.replace('*', ' ')
        line = line.replace(':', ' ')
        line = line.replace(',', ' ')
        line = line.replace('.', ' ')
        line = line.replace(')', ' ')
        line = line.replace('(', ' ')
        
        line = nonascii.sub('', line)
        line = measurers.sub('', line)
        line = descriptors.sub('', line)
        line = ammounts.sub('', line) 
        line = stopwords.sub('', line) 
        line = multiplespaces.sub(' ', line).strip()
        
        line = ' '.join( [w for w in line.split() if len(w)>1] )
        
        scrubbed_lines.append(line)
        
    scrubbed_ingredients.append(scrubbed_lines)

In [305]:
idx = 14
for i in range(len(ingredient_lists[idx])):
    print(f'{ingredient_lists[idx][i]:<60} | {scrubbed_ingredients[idx][i]}')

2 cups instant rice (such as Minute Rice)                    | instant rice such as minute rice
1/2 cup vanilla ice cream                                    | vanilla ice cream
1/2 teaspoon ground cinnamon, plus more for garnishing (optional) | cinnamon moregarnishing optional


In [306]:
from collections import Counter
vocab = Counter((line for ingredientlist in scrubbed_ingredients for line in ingredientlist))
sorted_vocab = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
len(sorted_vocab)

7457

In [307]:
sorted_vocab[:100]

[('salt', 447),
 ('olive oil', 280),
 ('eggs', 223),
 ('sugar', 207),
 ('flour', 187),
 ('water', 171),
 ('onion', 135),
 ('kosher salt', 119),
 ('butter', 115),
 ('vegetable oil', 92),
 ('cloves garlic', 85),
 ('freshly black pepper', 83),
 ('extra-virgin olive oil', 81),
 ('vanilla extract', 80),
 ('unsalted butter', 75),
 ('milk', 73),
 ('baking', 72),
 ('heavy cream', 69),
 ('egg', 68),
 ('pepper', 65),
 ('cumin', 64),
 ('garlic cloves', 63),
 ('clove garlic', 58),
 ('granulated sugar', 57),
 ('lemon juice', 56),
 ('black pepper', 54),
 ('baking soda', 53),
 ('soy sauce', 52),
 ('cinnamon', 51),
 ('fresh lemon juice', 51),
 ('garlic', 50),
 ('garlic clove', 48),
 ('scallions', 48),
 ('salt and pepper', 46),
 ('freshly pepper', 44),
 ('mayonnaise', 42),
 ('red onion', 41),
 ('fresh parsley', 39),
 ('cooking spray', 39),
 ('fresh cilantro', 38),
 ('whole milk', 38),
 ('kosher salt and freshly black pepper', 37),
 ('green onions', 37),
 ('sour cream', 35),
 ('chicken broth', 34),
 ('t

In [308]:
sorted_vocab[-100:]

[('tomato passata', 1),
 ('beaten egg brushing', 1),
 ('dark muscovado sugar', 1),
 ('saffron threads steeped in hot waterat least minutes', 1),
 ('the saffron custard', 1),
 ('the tomato sauce', 1),
 ('tomato pound', 1),
 ('seeded jalapeo pepper', 1),
 ('vegetable stock crumbled stock cube', 1),
 ('few basil leaves', 1),
 ('handfulbasil leaves optional', 1),
 ('tins plum tomato drained', 1),
 ('fresh garlic', 1),
 ('seeded peeled tomato', 1),
 ('low-sodium tomato juice', 1),
 ('basil leaves optional', 1),
 ('-inch-thick diagonally cut french bread baguette', 1),
 ('packetpizza dough mix made up', 1),
 ('capers rinsed and drained', 1),
 ('taleggio cheese', 1),
 ('-ounce part-skim mozzarella', 1),
 ('tintomato puree', 1),
 ('tinstomatoes', 1),
 ('tubmascarpone', 1),
 ('bunch fresh basil somegarnish', 1),
 ('tubsour cream', 1),
 ('roundedsalt', 1),
 ('sultana', 1),
 ('cooking apple', 1),
 ('light muscovado sugar', 1),
 ('litre jar spiced pickling vinegar', 1),
 ('-ounce part-skim mozzare

In [309]:
len(list(filter(lambda x: x[1] > 1, sorted_vocab)))

1418

In [310]:
flat_ingredients = []
for ilist in scrubbed_ingredients:
    doc = []
    for line in ilist:
        doc = doc + line.split(' ')
    flat_ingredients.append(doc)

In [311]:
scrubbed_ingredients[0]

['long-grain white rice', 'water', 'salt', 'olive oil']

In [312]:
flat_ingredients[0]

['long-grain', 'white', 'rice', 'water', 'salt', 'olive', 'oil']

In [313]:
vocab = Counter((word for doc in flat_ingredients for word in doc))
sorted_vocab = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
len(sorted_vocab)

4300

In [286]:
from gensim.models import Word2Vec, FastText

In [314]:
len(documents), len(labels)

(1827, 1827)

In [315]:
ft = FastText(
    flat_ingredients,
    size=32,
    window=5,
    min_count=3,
    workers=4
)

In [316]:
ft.wv.most_similar('butter')

[('buttermilk', 0.9999326467514038),
 ('buttercream', 0.9996895790100098),
 ('butternut', 0.9996778964996338),
 ('sugarcake', 0.9995439052581787),
 ('extractcake', 0.999491810798645),
 ('unsalted', 0.9994593858718872),
 ('pure', 0.9993953704833984),
 ('granulated', 0.9993525147438049),
 ('butternonstick', 0.9993157982826233),
 ('softened', 0.9992929697036743)]

In [317]:
ft.wv.most_similar('milk')

[('eggs', 0.9998264312744141),
 ('buttermargarine', 0.9997333288192749),
 ('sweetened', 0.9997109770774841),
 ('sifted', 0.9997015595436096),
 ('make', 0.9996931552886963),
 ('bittersweet', 0.9996913075447083),
 ('margarinebutter', 0.999682605266571),
 ('melted', 0.9996783137321472),
 ('icing', 0.9996709823608398),
 ('coconut', 0.9996532797813416)]

In [318]:
ft.wv.most_similar('tortilla')

[('chorizo', 0.9999201893806458),
 ('cream-filled', 0.9999183416366577),
 ('creamed', 0.9999127388000488),
 ('low-fatnonfat', 0.9999001026153564),
 ('strawberries', 0.999896228313446),
 ('raspberries', 0.9998951554298401),
 ('beaten', 0.9998926520347595),
 ('cooled', 0.9998918771743774),
 ('whole-wheat', 0.9998889565467834),
 ('cranberries', 0.9998863339424133)]

In [319]:
ft.wv.most_similar('sausage')

[('sausages', 0.9999462366104126),
 ('sage', 0.9998490810394287),
 ('pork', 0.9993568658828735),
 ('toast', 0.9993360042572021),
 ('-pound', 0.9993141889572144),
 ('least', 0.9992889165878296),
 ('breads', 0.9992820620536804),
 ('breakfast', 0.9992637634277344),
 ('breadcrumbs', 0.9992238283157349),
 ('best', 0.9992235898971558)]

In [320]:
ft.wv.most_similar('sugar')

[('sugarcake', 0.9999182224273682),
 ('granulated', 0.999729573726654),
 ('softened', 0.999713659286499),
 ('unsweetened', 0.999674916267395),
 ('extractcake', 0.9996375441551208),
 ('sodacake', 0.9996122717857361),
 ('extractthe', 0.999580979347229),
 ('chocolate', 0.999517023563385),
 ('cookies', 0.9994853734970093),
 ('cooking', 0.9994829893112183)]

In [ ]:
w2v = Word2Vec(
    
)

In [321]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(flat_ingredients)]
documents[0]

TaggedDocument(words=['long-grain', 'white', 'rice', 'water', 'salt', 'olive', 'oil'], tags=[0])

In [322]:
d2v = Doc2Vec(
    documents,
    vector_size=32,
    window=3,
    min_count=1,
    workers=4,
    epochs=40
)
d2v.train(documents, total_examples=d2v.corpus_count, epochs=model.epochs)

In [323]:
d2v.wv.most_similar(['butter'])

[('stick', 0.8460953235626221),
 ('sticks', 0.7687762975692749),
 ('seltzer', 0.7517319321632385),
 ('high-fateuropean-style', 0.7404323816299438),
 ('melted', 0.731892466545105),
 ('pecanswalnuts', 0.7221490144729614),
 ('dry-roasted', 0.7219871282577515),
 ('nutmegthe', 0.7142460942268372),
 ('flourpreparingpans', 0.71158766746521),
 ('optional;', 0.7050231099128723)]

In [325]:
d2v.wv.most_similar(['milk'])

[('substitute', 0.7625201940536499),
 ('eggsegg', 0.7280088663101196),
 ('lukewarm', 0.7253109812736511),
 ('skim', 0.7247210144996643),
 ('fat-free', 0.7209463715553284),
 ('milkbuttermilk', 0.6995106935501099),
 ('evaporated', 0.6931794881820679),
 ('lg', 0.6843627691268921),
 ('low-fat', 0.6775615215301514),
 ('carton', 0.6735019683837891)]

In [326]:
d2v.wv.most_similar(['tortilla'])

[('chef', 0.7315738797187805),
 ('fillo-pastry', 0.720894455909729),
 ('hard', 0.7158271670341492),
 ('crackers', 0.712313175201416),
 ('waterpickle', 0.704683244228363),
 ('salsachoice', 0.7031351327896118),
 ('marshmallows', 0.7019339799880981),
 ('pico', 0.697955846786499),
 ('refrigeratedjarred', 0.6955550312995911),
 ('warmed', 0.6953312158584595)]

In [324]:

model.infer_vector(documents[0].words)

array([ 0.01353001,  0.00711542, -0.01081877, -0.01772552, -0.00241778,
        0.0020362 ,  0.0029124 , -0.00906192,  0.01081262, -0.00150089,
       -0.00280596, -0.00920009, -0.01133201,  0.00698846, -0.00938813,
        0.00700257,  0.00222017, -0.00307623,  0.01473327,  0.00426297,
       -0.01005636,  0.01029589, -0.01853233,  0.00537654,  0.00495433,
       -0.00369038,  0.00647853, -0.00391025,  0.00168688, -0.00246186,
        0.01282562,  0.00964643], dtype=float32)